In [1]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
def asset_prediction(起始資金, 起始年紀,
    每月薪水 ,
    每月開銷 ,
    每月房租 ,
    退休年齡 ,
    投資部位,
    投資年利率,
    買房價格,
    買房頭期款,
    買房年紀,
    房貸利率,
    貸款年數,):
    
    預測時段 = range(起始年紀, 90)
    每年淨額 = pd.Series(0, index=預測時段)
    每年淨額.iloc[0] = 起始資金
    每年淨額.loc[:退休年齡] += 每月薪水 * 12
    每年淨額 -= (每月開銷 + 每月房租) *12
    
    
    def compound_interest(arr, ratio, return_rate):
        ret = [arr.iloc[0]]
        for v in arr[1:]:
            ret.append(ret[-1] * ratio * (return_rate/100 + 1) + ret[-1] * (1 - ratio) + v)
        return pd.Series(ret, 預測時段)
    
    買房花費 = pd.Series(0, index=預測時段)
    買房花費[買房年紀] = 買房頭期款
    買房花費.loc[買房年紀:買房年紀+貸款年數-1] += (買房價格 - 買房頭期款) / 貸款年數
    
    欠款 = pd.Series(0, index=預測時段)
    欠款[買房年紀] = 買房價格
    欠款 = 欠款.cumsum()
    欠款 = 欠款 - 買房花費.cumsum()
    利息 = 欠款.shift().fillna(0) * 房貸利率 / 100
    
    房租年繳 = pd.Series(每月房租*12, index=預測時段)
    房租年繳.loc[買房年紀:] = 0
    
    每年淨額_買房 = pd.Series(0, index=預測時段)
    每年淨額_買房.iloc[0] = 起始資金
    每年淨額_買房.loc[:退休年齡] += 每月薪水 * 12
    每年淨額_買房 -= (每月開銷*12 + 房租年繳 + 利息 + 買房花費)
    
    pd.DataFrame({
        'no invest, no house': 每年淨額.cumsum(),
        'invest, no house': compound_interest(每年淨額, 投資部位, 投資年利率),
        'no invest, house': 每年淨額_買房.cumsum(),
        'invest, house': compound_interest(每年淨額_買房, 投資部位, 投資年利率),
        
    }).plot()
    
    
    #import matplotlib.pylab as plt
    plt.ylim(0, None)
    
    print('月繳房貸', (買房價格 - 買房頭期款) / 貸款年數 / 12)
    print('利息', 利息.sum() / 貸款年數 / 12)
    print('總共月繳交: ', ((買房價格 - 買房頭期款) / 貸款年數 / 12)+(利息.sum() / 貸款年數 / 12))
    
import ipywidgets as widgets
widgets.interact(asset_prediction,
    起始資金=widgets.FloatSlider(min=0, max=100, step=10, value=20),
    起始年紀=widgets.IntSlider(min=0, max=100, step=1, value=30),
    每月薪水=widgets.FloatSlider(min=0, max=20, step=0.1, value=5.4),
    每月開銷=widgets.FloatSlider(min=0, max=20, step=0.1, value=1),
    每月房租=widgets.FloatSlider(min=0, max=20, step=0.1, value=1),
    退休年齡=widgets.IntSlider(min=0, max=100, step=1, value=60),
    投資部位=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.7),
    投資年利率=widgets.FloatSlider(min=0, max=30, step=0.1, value=5),
    買房價格=widgets.IntSlider(min=100, max=5000, step=50, value=300),
    買房頭期款=widgets.IntSlider(min=100, max=2000, step=10, value=100),
    買房年紀=widgets.IntSlider(min=20, max=100, step=1, value=40),
    房貸利率=widgets.FloatSlider(min=1, max=5, step=0.1, value=2.4),
    貸款年數=widgets.IntSlider(min=0, max=40, step=1, value=20)
)

interactive(children=(FloatSlider(value=20.0, description='起始資金', step=10.0), IntSlider(value=30, description=…

<function __main__.asset_prediction(起始資金, 起始年紀, 每月薪水, 每月開銷, 每月房租, 退休年齡, 投資部位, 投資年利率, 買房價格, 買房頭期款, 買房年紀, 房貸利率, 貸款年數)>

![table](table.png) ![pic](pic.png)